# Note:
This code is taken verbatim from the qiskit documentation. I have added comments and many explanatory sections to bolster and demonstrate my understanding of the code and of the underlying Quantum Mechanical / Chemistry phenomena which the code analyzes. Please see the other scripts in this repository to see how I've built upon this code to make it my own.

## Step 1 - Obtain Initial Hartree-Fock Solution

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

# Define the PySCF driver to simulate an H_2 molecule 
# with the hydrogen atoms distanced 0.735 ANGSTROM units apart
ANGSTROM_TO_BOHR = 1.8897259886
distance = 0.735
driver = PySCFDriver(
    atom=f"H 0 0 0; H 0 0 {distance}",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

In [ ]:
# Run the SCF driver to compute the electronic-hamiltonian for the system above.
# The hamiltonian is used in the eigenvalue minimization to find the ground state of the system.
problem = driver.run()
print(problem)

Show the terms in the 1 and 2-body integrals

$h_{pq} = \int \phi^*_p(r) \left( -\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{R_I- r} \right)   \phi_q(r)dr$
<br>
$h_{pqrs} = \int \frac{\phi^*_p(r_1)  \phi^*_q(r_2) \phi_r(r_2)  \phi_s(r_1)}{|r_1-r_2|}dr_1dr_2$


Here, we show the values of the integrals for electrons with a-spin. You can think of a-spin and b-spin as being "spin up" and "spin down".

The Hartree-Fock solution for the system's hamiltonian is defined as the following: <br>
$\hat{H}_{elec}=\sum_{pq} h_{pq} \hat{a}^{\dagger}_p \hat{a}_q + \frac{1}{2} \sum_{pqrs} h_{pqrs}  \hat{a}^{\dagger}_p \hat{a}^{\dagger}_q \hat{a}_r  \hat{a}_s$

The 1-body integrals describe the electron's energy without considering interactions with other electrons. This would include things like its kinetic energy.<br>
$h_{pq} = \int \phi^*_p(r) \left( -\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{R_I- r} \right)   \phi_q(r)dr$

The 2-body integrals describe energy resulting from interactions between bodies, like electron-electron interactions causing repulsion.<br>
$h_{pqrs} = \int \frac{\phi^*_p(r_1)  \phi^*_q(r_2) \phi_r(r_2)  \phi_s(r_1)}{|r_1-r_2|}dr_1dr_2$

In [10]:
hamiltonian = problem.hamiltonian

coefficients = hamiltonian.electronic_integrals
# Since H_2 is closed-shell, the alpha and beta coefficients should be the same. Here we inspect the alpha terms.
# The alpha-beta interactions in the 2-body integrals are accounted for in the hamiltonian, but we don't inspect them here since they're derived
# from the alpha and beta terms which are equal to eachother.
print(coefficients.alpha) 

Polynomial Tensor
 "+-":
array([[-3.25545567e+01,  5.37545363e-01,  3.30916626e-01,
        -1.54873042e-02,  2.82622528e-17, -1.01888574e-01,
        -1.15495181e-01],
       [ 5.37545363e-01, -7.45706391e+00, -8.63376441e-01,
         1.87170094e-01, -6.25340000e-16,  8.99390487e-01,
         5.23432955e-01],
       [ 3.30916626e-01, -8.63376441e-01, -6.02362473e+00,
        -3.57159396e-01,  1.17473775e-15, -1.09860325e+00,
        -2.89067083e-01],
       [-1.54873042e-02,  1.87170094e-01, -3.57159396e-01,
        -6.88326440e+00, -2.91602312e-15,  6.46255686e-01,
        -1.00187996e+00],
       [ 2.82622528e-17, -6.25340000e-16,  1.17473775e-15,
        -2.91602312e-15, -7.33984820e+00, -3.87382866e-15,
         4.96304685e-15],
       [-1.01888574e-01,  8.99390487e-01, -1.09860325e+00,
         6.46255686e-01, -3.87382866e-15, -6.23043487e+00,
        -8.00716832e-01],
       [-1.15495181e-01,  5.23432955e-01, -2.89067083e-01,
        -1.00187996e+00,  4.96304685e-15, -8.0071683

The following sections shows the second-quantized operators.

The second quantized operators are operators which describe the existence and movement of electrons in between spin orbitals.
When calculating the energy of the current state of a system, the hamiltonian - in second quantized form - represents energy coming from three distinct sources.

First, the effect of single electrons (with no interactions between them) are considered with the 2-term operators. Ex: $e$ * (+_0 -_0) contributes $e$ energy if there is an electron in spin orbital 0. The notation +_0 denotes a creation operator - where the hamiltonian creates an electron in the given orbital (0), while -_0 is an annihilation operator, removing an electron from the given orbital (0). Together, (+_0 -_0) removes, then creates an electron in the given orbital, resulting in no change. When calculating expectation value though, the term ends up contributing energy only if there is an electron in the given orbital since an annihilation operator on a state with no electron in the given orbital removes the term from the summation.

Terms with 4 operators (terms containing the two-body integrals) act similarly, but instead contribute energy originating from electron-electron interactions. The operators behave the same as when mentioned above, but now each term has 4 operators; 2 annihilation and 2 creation. Thus, the second source of energy represented in the hamiltonian is these electron-electron interactions represented as terms with 4 operators. Some terms (diagonal terms) are essentially the same as the single body terms. For example ( +_0 +_1 -_1 -_0 ) represents removing and then replacing electrons in the 0 and 1 spin orbitals. This clearly has no effect in terms of moving electrons between orbitals, and effectively just adds the energy associated with these terms if there are electrons found in the 0 and 1 orbitals. This is energy originating from the repulsion of the two electrons in the 0 and 1 orbitals.

Finally, some terms with 4 operators - like ( +_0 +_2 -_3 -_1 ) represent evolutions where electrons move between orbitals. When calculating the expectation value, these terms only persist and add their corresponding energy values if some electrons are in a superposition of multiple orbitals. This is because during the calculation of the expectation value, we take the inner product of the new state with the original state. If the new state has these electrons scattered into different orbitals, the state will be orthogonal with the original, negating any energy that may have resulted from this new scattered state. So, these scattering operators capture the energy originating from delocalization.

### Simplified Example: 
Let $\ket{0, 1}$ represent a state where orbital 0 is unoccupied, and electron 1 is occupied, with $\ket{1, 0}$ representing the opposite.

if $\ket{\psi} = \ket{0, 1}$ and we look at the operator $\hat{O} = +_0-_1$, then we get the following:

$\hat{O}\ket{\psi} = \ket{1, 0}$, so the expectation value of this state for this observable is $\bra{0, 1}\hat{O}\ket{0, 1} = \bra{0, 1}\ket{1, 0} = 0$

However, with a state in a superposition $\ket{\psi\prime} = \frac{1}{\sqrt{2}}(\ket{0, 1} + \ket{1, 0})$, we get the following:

$\bra{\psi\prime}\hat{O}\ket{\psi\prime} = \frac{1}{2}(\bra{0, 1} + \bra{1, 0})(\hat{O}\ket{0, 1} + \hat{O}\ket{1, 0}) = \frac{1}{2}(\bra{0, 1} + \bra{1, 0})(\hat{O}\ket{0, 1} + 0) = \frac{1}{2}(\bra{0, 1} + \bra{1, 0})(\ket{1, 0}) = \frac{1}{2}$

In [11]:
second_q_op = hamiltonian.second_q_op()
print(second_q_op)

Fermionic Operator
number spin orbitals=14, number terms=6126
  -32.55455670213795 * ( +_0 -_0 )
+ 0.5375453625745057 * ( +_0 -_1 )
+ 0.33091662594346577 * ( +_0 -_2 )
+ -0.015487304195897474 * ( +_0 -_3 )
+ -0.10188857436447904 * ( +_0 -_5 )
+ -0.11549518099952288 * ( +_0 -_6 )
+ 0.537545362574506 * ( +_1 -_0 )
+ -7.457063909908603 * ( +_1 -_1 )
+ -0.863376441055675 * ( +_1 -_2 )
+ 0.18717009370599333 * ( +_1 -_3 )
+ 0.8993904871763646 * ( +_1 -_5 )
+ 0.5234329553378084 * ( +_1 -_6 )
+ 0.33091662594346427 * ( +_2 -_0 )
+ -0.8633764410556755 * ( +_2 -_1 )
+ -6.023624733545527 * ( +_2 -_2 )
+ -0.35715939609271496 * ( +_2 -_3 )
+ -1.098603253764612 * ( +_2 -_5 )
+ -0.2890670826844037 * ( +_2 -_6 )
+ -0.015487304195897418 * ( +_3 -_0 )
+ 0.18717009370599283 * ( +_3 -_1 )
+ -0.3571593960927155 * ( +_3 -_2 )
+ -6.883264401254342 * ( +_3 -_3 )
+ 0.6462556862623549 * ( +_3 -_5 )
+ -1.001879961463841 * ( +_3 -_6 )
+ -7.339848196505165 * ( +_4 -_4 )
+ -0.10188857436447894 * ( +_5 -_0 )
+ 0.8993

## Step 2 - Compute the ground state energy level of $H_2$
Now that we have Hartree-Fock Hamiltonian of the $H_2$ molecule, we can use qiskit's ground state eigensolver to compute / approximate the ground state energy level.

In [12]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [ ]:
# Solve for the minimum eigenvalue of the hamiltonian
result = solver.solve(problem)

# The Qiskit ground state eigensolver doesn't automatically dd nuclear repulsion energy.
# Manually add it to the result
dist_bohr = distance * ANGSTROM_TO_BOHR
nuclear_repulsion_energy = 1/dist_bohr

print(result.eigenvalues + nuclear_repulsion_energy)

[-83.18577764-5.40374119e-16j]
